# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import sys
from ipywidgets.embed import embed_minimal_html,embed_data
# Import API key
sys.path.insert(1, '..')
from api_keys import g_key
gmaps.configure(api_key=g_key)
pd.options.mode.chained_assignment = None  # default='warn'

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
CityWeathDF = pd.read_csv('../WeatherPy/output.csv')
CityWeathDF

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,4030556,rikitea,0,PF,1603139767,78,-23.12,-134.97,74.14,12.82
1,2063036,port lincoln,100,AU,1603139878,86,-34.73,135.87,55.89,12.62
2,3833367,ushuaia,90,AR,1603139753,80,-54.80,-68.30,39.20,38.03
3,3369157,cape town,40,ZA,1603139878,67,-33.93,18.42,61.00,10.29
4,5880054,barrow,90,US,1603139362,85,71.29,-156.79,21.20,10.29
...,...,...,...,...,...,...,...,...,...,...
550,1524298,aksu,83,KZ,1603140561,74,52.03,76.92,28.40,8.95
551,119505,qazvin,0,IR,1603140561,29,36.28,50.00,51.80,4.70
552,1513957,hazorasp,0,UZ,1603140561,37,41.32,61.07,50.00,2.24
553,2026861,berdigestyakh,100,RU,1603140561,95,62.10,126.70,14.88,5.57


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Store latitude and longitude in locations
loc = CityWeathDF[["Lat", "Lng"]]
Templvl = CityWeathDF["Max Temp"].astype(float)

In [14]:
#Graph
HighestHumid = CityWeathDF['Humidity'].max()
fig = gmaps.figure()
HeatLay = gmaps.heatmap_layer(loc, weights=Templvl, dissipating=False, max_intensity=HighestHumid, point_radius=3)
fig.add_layer(HeatLay)
embed_minimal_html('Images/HumidityHeatmap.html', views=[fig])

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
IdealCityTemp = CityWeathDF.loc[(CityWeathDF.Humidity <50)&(CityWeathDF['Max Temp'] < 75)&(CityWeathDF['Max Temp'] > 60)&(CityWeathDF['Wind Speed'] < 15)&(CityWeathDF['Cloudiness'] > 20)]
IdealCityTemp = IdealCityTemp.reset_index(drop=True)
IdealCityTemp

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,2065594,mount isa,35,AU,1603139948,18,-20.73,139.50,73.40,4.70
1,3901903,uyuni,83,BO,1603140137,25,-20.46,-66.83,64.31,14.07
2,1011632,de aar,59,ZA,1603140356,25,-30.65,24.01,67.75,10.29
3,5703673,elko,56,US,1603140428,26,41.00,-115.50,73.00,11.99


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = IdealCityTemp.drop(columns=['Cloudiness', 'Date', 'Humidity', 'Max Temp', 'Wind Speed'])
hotel_df

,City_ID,City,Country,Lat,Lng
0,2065594,mount isa,AU,-20.73,139.50
1,3901903,uyuni,BO,-20.46,-66.83
2,1011632,de aar,ZA,-30.65,24.01
3,5703673,elko,US,41.00,-115.50


In [7]:
#Converting hoteldf into list
CityList = [city for city in hotel_df['City']]
LatList = [city for city in hotel_df['Lat']]
LngList = [city for city in hotel_df['Lng']]
CountryList = [city for city in hotel_df['Country']]

#Defining lists of info grabbed
FirstHotelName = []
HotelCity = []
FirstHotelLat = []
FirstHotelLong = []

for city in CityList:
    target_type = "lodging"
    cityloc = f'{LatList[CityList.index(city)]},{LngList[CityList.index(city)]}'
    radius = 15000
    params = {
        "location": cityloc,
        "types": target_type,
        "radius": radius,
        "key": g_key
    }
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"


    response = requests.get(base_url, params)
    hotel = response.json()
    FirstHotelName.append(hotel['results'][0]['name'])
    HotelCity.append(hotel['results'][0]['vicinity'])
    FirstHotelLat.append(hotel['results'][0]['geometry']['location']['lat'])
    FirstHotelLong.append(hotel['results'][0]['geometry']['location']['lng'])
hotel_df = pd.DataFrame({'Name': FirstHotelName, 'Country': CountryList, 'City': HotelCity, 'Lat': FirstHotelLat, 'Lng': FirstHotelLong})

In [8]:
# NOTE: Do not change any of the code in this cell
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [9]:
fig1 = gmaps.figure()
HeatLay = gmaps.heatmap_layer(locations, dissipating=False, max_intensity=10, point_radius=3)
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig1.add_layer(HeatLay)
fig1.add_layer(markers)
embed_minimal_html('Images/HotelMap.html', views=[fig1])